# Apllication of ScAsAT 

This python notebook introduces __S__ingle-__c__ell __A__TAC-seq __A__nalysis __T__ool (ScAsAT). We applied this tool to analyze the human data generated in the paper by [Buenrostro et. al](http://www.nature.com/nature/journal/v523/n7561/full/nature14590.html). Although they have nouse and human data, where we showed the application of only human data.

### Import python packages

In [1]:
import subprocess, os, csv, signal, pysam, rpy2
import numpy as np
import readline
import rpy2.robjects
import matplotlib.pyplot as plt
import csv
from operator import itemgetter
%matplotlib inline

This code toggles between showing/hiding the codes. The outputs and the discussions would always be visible.

In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



rpy2 package provides a simple and robust access to `R` from within `Python`. The following cells loads the package to run `R` code within `Python` notebook. To write and execute `R` code we will use the magic command `%%R` at the top of the cell. 

In [6]:
%load_ext rpy2.ipython

## Python Functions

In [5]:
def formatSummitBed(summitFile, fileToSave):
#fileToSave = '/home/baker/my-dpsf-scratch-share/Conor_Rogerson_Older_Genome
#                /output/Bam_Files_Filtered_on_Library_Size/AllFiles_Filtered_summits_shifted.bed' 
#peakFile = '/home/baker/my-dpsf-scratch-share/Conor_Rogerson_Older_Genome
#            /output/Bam_Files_Filtered_on_Library_Size/AllFiles_Filtered_summits.bed'

    with open(summitFile,'r') as f:
        peaks=[x.strip().split('\t') for x in f]
    for i in range(len(peaks)):
        peaks[i][1] = str(int(peaks[i][1]) - 250)
        peaks[i][2] = str(int(peaks[i][2]) + 250)

    peaks.sort(key=lambda x: float(x[4]), reverse=True)

    with open(fileToSave, "w") as f:
        writer = csv.writer(f,delimiter='\t')
        writer.writerows(peaks)

`formatSummitBed()`: 
> This functions formats the summit file by adding 500 bp distance from the summit position.

Arguments:
> `summitFile` : The name of the summit file. This has to be the file that is generated from the aggreagated `BAM` file<br>
> `fileToSave`: The name of the file where the save the peak file with the 500 bp peaks.

Value:
> File with 500 bp peaks.

In [7]:
def getMergedBam(BAMFolder, BAMFilePattern):
    
    import pandas as pd
    
        
    mergeFiles = ''
    for root, folder, files in os.walk(BAMFolder):
            files = [os.path.join(root, f) for f in files if (f.endswith(BAMFilePattern))]
            for f in files:
                mergeFiles = mergeFiles +  f + ' '
    
        
    mergeCommand = 'samtools merge ' + BAMFolder + 'AllFiles_Filtered_Merged_nodup.bam' + ' ' + mergeFiles    
    subprocess.call(mergeCommand, shell=True)   
    
    
    sortMergeCmd = 'samtools sort ' + BAMFolder + 'AllFiles_Filtered_Merged_nodup.bam' \
                    + ' ' + BAMFolder + 'AllFiles_Filtered_Merged_nodup_sorted'        
    subprocess.call(sortMergeCmd, shell=True)
    
    indexMergeCmd = 'samtools index ' + BAMFolder + 'AllFiles_Filtered_Merged_nodup_sorted.bam'
    subprocess.call(indexMergeCmd, shell=True)

`getMergedBam()`:
> merges all the individual `BAM` files for each of the cells and creates `AllFiles_Filtered_Merged_nodup.bam` file. This file is then sorted and indexed.

Arguements:
> `BAMFolder`: Location of the folder where all the BAM files for each of the single-cells are. <br>
> `BAMFilePattern` : The end pattern of the `BAM` files to be merged.

Value:
> The merged `BAM` file `AllFiles_Filtered_Merged_nodup_sorted.bam`. 

In [8]:
def getAggregatedPeak(BAMFolder, BAMFilePattern):
    getMergedBam(BAMFolder, BAMFilePattern)
    
    MergedMacs2Cmd = 'macs2 callpeak -t ' + BAMFolder + 'AllFiles_Filtered_Merged_nodup_sorted.bam' +' -n ' \
                    + BAMFolder + 'AllFiles_Filtered' +' -q 0.01 -g hs -f BAM --nomodel --nolambda --shift -75 --extsize 150 \
                    -B --SPMR --call-summits --keep-dup all'
    subprocess.call(MergedMacs2Cmd, shell=True)
    
    narrowPeakFile = BAMFolder + 'AllFiles_Filtered_peaks.narrowPeak'
    mergedPeakFile = BAMFolder + 'mergedPeaks.bed'
    
    
    with open(narrowPeakFile,'r') as f:
        peaks=[x.strip().split('\t') for x in f]
    peaks.sort(key=lambda x: float(x[8]), reverse=True)
    
    
    peaksToWrite = list()
    for i in range(len(peaks)):
        peaksToWrite.insert(i, peaks[i][0:4])
        peaksToWrite[i].append(peaks[i][8])      
    
    
    with open(mergedPeakFile, "w") as f:
        writer = csv.writer(f,delimiter='\t')        
        writer.writerows(peaksToWrite)        

`getAggregatedPeak()`:
> This function calls `MACS2` on the aggregated `BAM` file and generates the peaks. If the user is following the merging BAM option then this is the function they need to call. Then the `getMergedBam()` is called within this function

Arguements:
> `BAMFolder`: Location of the folder where all the BAM files for each of the single-cells are. <br>
> `BAMFilePattern` : The end pattern of the `BAM` files to be merged.

Value:
> Peak files from MACS2 call.

------------------
## R functions

### Loading R packages (if not available it will install it before loading)

In [10]:
%%R
if (!require("pacman")) install.packages("pacman")
pacman::p_load(ggplot2, ComplexHeatmap, Rtsne, prabclus, GGally, rtracklayer, VGAM, JASPAR2016, TFBSTools, motifmatchr)

### R functions for the ScAsAT

In [11]:
%%R
peakAccesibility <- function(mergedPeakFile, peakFolder, peakFilePattern, outputFolder, outputPeakFileName){
    if(length(list.files(path=peakFolder, pattern = paste0(peakFilePattern,'.bed'), full.names=TRUE))>0){
        files = list.files(path=peakFolder, pattern = paste0(peakFilePattern,'.bed'), full.names=TRUE)
        unlink(files)
    }
    formatingToBed(peakFolder, peakFilePattern, outputFolder)
    peakFilePattern = paste0(peakFilePattern,'.bed')
    query = import(mergedPeakFile)
    queryDF <- data.frame(query)
    #queryDF <- queryDF[order(queryDF$name, decreasing=TRUE),]    
    
    totalOverlap <- data.frame(seqnames = queryDF$seqnames, start = queryDF$start, end = queryDF$end)    
    files <- list.files(path=peakFolder, pattern = peakFilePattern, full.names=TRUE)
    cellName <- list.files(path=peakFolder, pattern = peakFilePattern)
    for (i in 1:length(files)){
        subject = import(files[i])
        hits = findOverlaps(query, subject)
        hitsDF <- data.frame(hits)
        cellName[i] <- gsub(peakFilePattern, '', cellName[i])
        totalOverlap[hitsDF$queryHits, cellName[i]] <- 1
        totalOverlap[-hitsDF$queryHits, cellName[i]] <- 0
    }
    outputFile = paste0(outputFolder,outputPeakFileName)
    write.table(totalOverlap, outputFile, row.names=FALSE)    
}    


`peakAccesibility()`:
> This function calculates the accessible peaks for each of the cells. It looks at each of the peaks listed in _Aggregated-Peak_ table and tries to find out whether there is any overlap between these peaks and the peaks of each cell. If there is then it puts a 1 agains that cells for that peak. Otherwise it puts 0. Calling these function for each of the cell completes the _Aggregated-Peak_ for which rows represnt all the peaks and the column represents cells.

Arguements:
> `mergedPeakFile`: This is the name and location of `mergedPeaks.bed` file that were generated while calling `mergePeaks()` python function.<br>
> `peakFolder`: This is the location of the folder where all the peaks for each of the single-cells are located. <br>
> `peakFilePattern` : The end pattern of the `peak` files of each single cells found in the `peakFolder`. Currently the only two accepted patterns are `_peaks.narrowPeak` and `_summits_shifted.bed` <br>
> `outputFolder`: Folder where all the outputs will be saved
> `outputPeakFileName`: Name of the file where the peak accessibility table, _Aggregated-Peak_ would be saved. This file will contain all the accessibility information for all the cells.


Value:
> Peak accessibile table saved according to the name set in `outputPeakFileName`. The table will have the results in 1 or 0 expression peak opening or closing respectively.

In [12]:
%%R
formatingToBed <- function(peakFolder, peakFilePattern, outputFolder){
    files <- list.files(path=peakFolder, pattern = peakFilePattern, full.names=TRUE)
    for (i in 1:length(files)){
        narrowPeak <- read.csv(files[i], header=FALSE, sep='\t')
        write.table(narrowPeak[,1:4], paste0(files[i],'.bed'), 
                    row.names=FALSE, sep='\t', col.names=FALSE, quote=FALSE)
    }
}

`formatingToBed()`:
> This function formats the bed file suitable for processing by ScAsAT. It adds a `.bed` extension at the end.

Arguements:
> `mergedPeakFile`: This is the name and location of `mergedPeaks.bed` file that were generated while calling `mergePeaks()` python function.<br>
> `peakFolder`: This is the location of the folder where all the peaks for each of the single-cells are located. <br>
> `peakFilePattern` : The end pattern of the `peak` files of each single cells found in the `peakFolder`. Currently the only two accepted patterns are `_peaks.narrowPeak` and `_summits_shifted.bed` <br>
> `outputFolder`: Folder where all the outputs will be saved

Value:
> Formats the bed files with adding a `.bed` extesion.

In [13]:
%%R
getDiffAccessLogsitic <- function(cdBinary, groups=NULL){

    if (length(levels(groups)) != 2) {
        stop(paste("ERROR: wrong number of levels in the grouping factor (", 
            paste(levels(groups), collapse = " "), "), but must be two.", 
            sep = ""))
    }
    if (is.null(groups)){
        stop("ERROR: groups factor is not provided")
    }
    
    
    SingleCell.Group1.CellNames <- names(groups[groups==levels(groups)[1]])
    SingleCell.Group2.CellNames <- names(groups[groups==levels(groups)[2]])
    
    SingleCell.Group1.Binary <- cdBinary[,SingleCell.Group1.CellNames]
    SingleCell.Group2.Binary <- cdBinary[,SingleCell.Group2.CellNames]
    
    CellType <- data.frame(CellType=gsub('_.*','',names(groups)))
         
    SingleCell.Group1VsGroup2 <- cbind(SingleCell.Group1.Binary,SingleCell.Group2.Binary)
    SingleCell.Group1VsGroup2 <- t(SingleCell.Group1VsGroup2)
    SingleCell.Group1VsGroup2 <- cbind(SingleCell.Group1VsGroup2,CellType)   
    
    dataDim = (dim(SingleCell.Group1VsGroup2)[2]-1)
    #dataDim = 10
    
    pvalue = vector(mode="numeric", length=(ncol(SingleCell.Group1VsGroup2)-1))
    SingleCell.Group1VsGroup2.res <- data.frame(Chr = cdBinary[1:dataDim,1],
                                           Start = cdBinary[1:dataDim,2],
                                           end = cdBinary[1:dataDim,3])        
    
    
    for(i in 1:dataDim)
    {
        hfit1 <- vgam(SingleCell.Group1VsGroup2[,i] ~ SingleCell.Group1VsGroup2$CellType, binomialff) 
        res <- VGAM::lrtest(hfit1, "SingleCell.Group1VsGroup2$CellType")        
        SingleCell.Group1VsGroup2.res$pvalue[i] <- res@Body[5][2,]        
    }
    
        
    SingleCell.Group1.OneCount <- rowSums(SingleCell.Group1.Binary==1)
    SingleCell.Group2.OneCount <- rowSums(SingleCell.Group2.Binary==1)
    
    # Bonferroni correction
    SingleCell.Group1VsGroup2.res$p_adjust <- p.adjust(SingleCell.Group1VsGroup2.res$pvalue, method="hochberg")
    #print(p.adjust(pvalue, method="bonferroni", n=(dim(SingleCell.Group1VsGroup2)[2]-1)))
    #print(pvalue)

    
    
    write.csv(SingleCell.Group1VsGroup2[order(SingleCell.Group1VsGroup2$pvalue, decreasing=FALSE),], 
              paste0(levels(groups)[1],'_vs_',levels(groups)[2],'Logistic.csv'), row.names=FALSE)
    
    return(SingleCell.Group1VsGroup2.res)    
}

`getDiffAccessLogsitic()`:
> Caculates differential accessibility between two groups of cells using logistic regression. For each peak the accessibility is encoded as a binary variable. The group each cell belongs to is used as a catagorical preidctor variable for accessiblity. A generalized linear additive model (GLAM) is used to model the accessibility with the predictor variable. Likelihood ratio test is used to test the statistical significance of observed differences in each site's accessibility due the group of the cells. Benjamini and Hochberg method is used to correct the p-values for multiple testing. Finally the function returns a table with the peaks, the p-value and the q-value (corrected p-value).


`Usage`:
> `getDiffAccessLogsitic(count_data, groups)`

Arguements:
> `cdBinary`: The peak information converted into binary. Ideally this would be file that set in the `outputPeakFileName`. `cdBinary` would be used to calculate the Jaccard distance which would then be used as d matrix in MDS calculation<br>

>`groups` : groups into which each cell belong to.

Value:
> Returns a table with the peaks, the p-value and the q-value (corrected p-value).

In [14]:
%%R
getDiffAccessLInformationGain <- function(cdBinary, groups=NULL){

    if (length(levels(groups)) != 2) {
        stop(paste("ERROR: wrong number of levels in the grouping factor (", 
            paste(levels(groups), collapse = " "), "), but must be two.", 
            sep = ""))
    }
    if (is.null(groups)){
        stop("ERROR: groups factor is not provided")
    }
    
    
    SingleCell.Group1.CellNames <- names(groups[groups==levels(groups)[1]])
    SingleCell.Group2.CellNames <- names(groups[groups==levels(groups)[2]])
    
    SingleCell.Group1.Binary <- cdBinary[,SingleCell.Group1.CellNames]
    SingleCell.Group2.Binary <- cdBinary[,SingleCell.Group2.CellNames]
    
    CellType <- data.frame(CellType=gsub('_.*','',names(groups)))
    
         
    SingleCell.Group1VsGroup2 <- cbind(SingleCell.Group1.Binary,SingleCell.Group2.Binary)
    SingleCell.Group1VsGroup2 <- t(SingleCell.Group1VsGroup2)
    #SingleCell.Group1VsGroup2 <- cbind(SingleCell.Group1VsGroup2,CellType)   

    
    dataDim = (dim(SingleCell.Group1VsGroup2)[2])
    #dataDim = 5
    
    information.gain = vector(mode="numeric", length=(ncol(SingleCell.Group1VsGroup2)-1))
    SingleCell.Group1VsGroup2.res <- data.frame(Chr = cdBinary[1:dataDim,1],
                                           Start = cdBinary[1:dataDim,2],
                                           end = cdBinary[1:dataDim,3])        
    
    
    for(i in 1:dataDim)
    {
        jointData = cbind(SingleCell.Group1VsGroup2[,i], CellType)
        
        gain <- calcEntropy(jointData[,2]) - 
                (sum(table(jointData)[c(1,3)])/length(jointData[,2]))*calcEntropy(jointData[jointData[,1]==0,]) - 
                (sum(table(jointData)[c(2,4)])/length(jointData[,2]))*calcEntropy(jointData[jointData[,1]==1,])
        SingleCell.Group1VsGroup2.res$information.gain[i] <- gain        
    }
    
        
    SingleCell.Group1.OneCount <- rowSums(SingleCell.Group1.Binary==1)
    SingleCell.Group2.OneCount <- rowSums(SingleCell.Group2.Binary==1)
    

    #write.csv(SingleCell.Group1VsGroup2[order(SingleCell.Group1VsGroup2$information.gain, decreasing=TRUE),], 
    #          paste0(levels(groups)[1],'_vs_',levels(groups)[2],'InformationGain.csv'), row.names=FALSE)
    
    return(SingleCell.Group1VsGroup2.res)    
}

`getDiffAccessLInformationGain()`:
> Caculates differential accessibility between two groups of cells using entropy and information gain. For each peak the accessibility is encoded as a binary variable. For each peak we calculate the "Information gain" by having the peak divided into two groups of cells for which we are trying to find out the differential accessibilty. The Information gain is calculated as

$$ 
 Gain(P, P_{G_1}, P_{G_2}) = Entropy(P) - \sum_{v \epsilon\{G_1,G_2 \}} \frac{|P_v|}{|P|}Entropy(P_v)
$$

>where Entropy is calculated as
$$\sum_{i=1}^c -p_i log_2 p_i$$

> The peaks are then sorted based on the informatin gain and the user can choose the cutoff value for selecting the DA peaks. A suggested value for this would be values < 0.01


`Usage`:
> `getDiffAccessLogsitic(count_data, groups)`

Arguements:
> `cdBinary`: The peak information converted into binary. Ideally this would be file that set in the `outputPeakFileName`. `cdBinary` would be used to calculate the Jaccard distance which would then be used as d matrix in MDS calculation<br>

>`groups` : groups into which each cell belong to.

Value:
> Returns a table with the peaks along with Information gain.

In [15]:
%%R
getPeakVsPrecision <- function(peakA, peakB,start=100,stepSize=100){
    
    dimToRun <- min(dim(peakA)[1],dim(peakB)[1])
    if(dim(peakB)[1]>dimToRun){
        peakB <- peakB[1:dimToRun,]
    }
    GenomicRangeObjA <- makeGRangesFromDataFrame(peakA,
                         keep.extra.columns=FALSE,
                         ignore.strand=TRUE,
                         seqinfo=NULL,
                         seqnames.field=c("Chr"),
                         start.field="Start",
                         end.field=c("End"),
                         starts.in.df.are.0based=FALSE)
    j=1
    Recall <- vector(mode="numeric", length=0)
    Precision <- vector(mode="numeric", length=0)
    for(i in seq(100,dimToRun,stepSize)){
           GenomicRangeObjB <- makeGRangesFromDataFrame(peakB[1:i,],
                         keep.extra.columns=FALSE,
                         ignore.strand=TRUE,
                         seqinfo=NULL,
                         seqnames.field=c("Chr"),
                         start.field="Start",
                         end.field=c("End"),
                         starts.in.df.are.0based=FALSE)
                         
           intersectRes <- as.data.frame(intersect(GenomicRangeObjA,GenomicRangeObjB))
           FP = dim(peakB[1:i,])[1] - dim(intersectRes)[1]
           TP = dim(intersectRes)[1]
        
           GenomicRangeObjB <- makeGRangesFromDataFrame(peakB[(i+1):dim(peakB)[1],],
                         keep.extra.columns=FALSE,
                         ignore.strand=TRUE,
                         seqinfo=NULL,
                         seqnames.field=c("Chr"),
                         start.field="Start",
                         end.field=c("End"),
                         starts.in.df.are.0based=FALSE)
                         
           intersectRes <- as.data.frame(intersect(GenomicRangeObjA,GenomicRangeObjB))
           TN = dim(peakB[(i+1):dim(peakB)[1],]) - dim(intersectRes)[1]
           FN = dim(intersectRes)[1]
           Recall[j] = 100*(TP/(TP + FN))
           Precision[j] = 100*(TP / (TP + FP))
           j = j +1
           
   
    }
    df<-data.frame(x=Recall,y=Precision)
    p <- ggplot(df, aes_string(x="x",y ="y"))+ scale_colour_hue(l=40)
    
    p <- p + ggtitle("Recall vs Precision") + theme(plot.title = element_text(size = 16, face = "bold"))
    p <- p + geom_point(size = 2)
    p <- p + xlab("Recall%") 
    p <- p + ylab("Precision%")+
	theme_light(base_size=15) +
        theme(strip.background = element_blank(),
              panel.border     = element_blank(),
              plot.title = element_text(hjust = 0.5))
    print(p)
    
    GenomicRangeObjB <- makeGRangesFromDataFrame(peakB,
                         keep.extra.columns=FALSE,
                         ignore.strand=TRUE,
                         seqinfo=NULL,
                         seqnames.field=c("Chr"),
                         start.field="Start",
                         end.field=c("End"),
                         starts.in.df.are.0based=FALSE)
                         
   intersectRes <- as.data.frame(intersect(GenomicRangeObjA,GenomicRangeObjB))
   
   return(intersectRes) 
}

`getPeakVsPrecision()`:
> This function is used to see how many peaks from the Bulk data the single-cell have been able to capture. For that we calculate the Peak and the Precision of the single-cell data against its Bulk counterpart and then plot it. The way these two values are calculated are

$$
Recall = 100  \frac{TP}{TP + FN}
$$
$$
Precision = 100  \frac{TP}{TP + FP}
$$
> where <br>
> TP = True Positive <br>
> TN = True Negative <br>
> FP = False Positive <br>
> FN = False Negative <br>

`Usage`:
> `getPeakVsPrecision(peakA, peakB,start,stepSize)`

Arguements:
> `peakA`: Peak of the BULK data <br>
> `peakB`: Single-cell Peak <br>
> `start`: Number of peaks to start with the calculation, default is `100`. <br>
> `stepSize`: Size of the increase steps

Value:
> Plots Recall vs. Precision and returns a summary

In [16]:
%%R
getJaccardDist <- function(cdBinary){
        
    if(colnames(cdBinary[,2:3])[1] == 'start' && colnames(cdBinary[,2:3])[2] == 'end'){
        SingleCell.Binary <- cdBinary[,4:(dim(cdBinary)[2])]
    }
    else
        SingleCell.Binary <- cdBinary
    
    
    SingleCell.Binary.Jaccard <- jaccard(as.matrix(SingleCell.Binary))
    
    return(SingleCell.Binary.Jaccard)
}

`getJaccardDist()`:
> This function calculates the jaccard distance between each of the cells. Jaccard distance calculates the ratio between the differences of the sizes of the union and intersection against the size of the union. The equation for calculating this distance is <br><br>$$d_J(A,B)=\frac{|A \cup B| - |A \cap B|}{|A \cup B|}$$

Arguements:
> `cdBinary`: The peak information converted into binary. Ideally this would be file that set in the `outputPeakFileName`. 


Value:
> Jaccard distance between each of the cell.

In [17]:
%%R
plotCluster <- function(cdBinary, k, groups=NULL, cellName, ret.val=FALSE, text.label=FALSE){

    if(missing(k)){
        stop("ERROR: Number of groups \"k\" is missing")
    }
    if(missing(ret.val)){
        ret.val = FALSE
    }
    if(text.label==TRUE & missing(cellName)){
        stop("ERROR: Please give cellName")
    }
    
    SingleCell.Binary.Jaccard <- getJaccardDist(cdBinary)
    fit <- cmdscale(as.dist(SingleCell.Binary.Jaccard),eig=TRUE, k=k)
 
    if(is.null(groups)){
        df<-data.frame(x=fit$points[,1],y=fit$points[,2], Cell=colnames(SingleCell.Binary.Jaccard))
        p <- ggplot(df, aes_string(x="x",y ="y"))+ scale_colour_hue(l=40)
    }
    else{
        df<-data.frame(x=fit$points[,1],y=fit$points[,2], Cell=colnames(SingleCell.Binary.Jaccard), Batch=groups)
        p <- ggplot(df, aes_string(x="x",y ="y", color="Batch"))+ scale_colour_hue(l=40)
    }

    
    p <- p + ggtitle("MDS with Jaccard Distance Matrix") + theme(plot.title = element_text(size = 16, face = "bold"))
    p <- p + geom_point(size = 2)
    p <- p + xlab("Corrdinate 1") 
    p <- p + ylab("Corrdinate 2")+
	theme_light(base_size=15) +
        theme(strip.background = element_blank(),
              panel.border     = element_blank(),
              plot.title = element_text(hjust = 0.5))

    if(length(levels(groups)) < 12)
        p <- p + scale_colour_brewer(palette="Set1")
    if(text.label==TRUE)
        p <- p + geom_text(data=df,aes(label=cellName),alpha=0.5,size=4, vjust=1,hjust=0.5,angle=45)

    #p <- p + theme(axis.title = element_text(size = 14), axis.text = element_text(size = 14),
    #legend.text = element_text(size = 14), legend.title = element_text(size = 14))
    
    print(p)
    
    if(ret.val == TRUE)
        return(fit)
}

`plotCluster()`:
> This function calculates the Multidimensional Scaling (MDS) and plots the cells based on Jaccard distance to visualize the level of similarity between the cells.

Arguements:
> `cdBinary`: The peak information converted into binary. Ideally this would be file that set in the `outputPeakFileName`. `cdBinary` would be used to calculate the Jaccard distance which would then be used as d matrix in MDS calculation<br>
`k`: The maximum dimension of the space which the data are to be represented in; must be in {1, 2, ..., n-1}. <br>
`groups`: If the cells can be organized into different groups (for eg. different batches), then they are passed as groups making them factor variable.<br>
`cellName`: If `text.label` is set to `TRUE` then you have to supply `cellName`.
`ret.val`: Need to set this TRUE for the function to return the MDS values <br>
`text.label`: Need to set this TRUE if label of each of the cells are to be printed in the plot.


Value:
> Calculates the MDS and plots the cells based on their MDS dimension. It returns the MDS values if `ret.val` is set to TRUE.

In [18]:
%%R
plotPCAJaccard <- function(cdBinary, groups=NULL, ret.val=FALSE , cellName, text.label=FALSE, title){
    
   
    if(missing(ret.val)){
        ret.val = FALSE
    }
    if(text.label==TRUE & missing(cellName)){
        stop("ERROR: Please give cellName")
    }
    
    SingleCell.Binary.Jaccard <- getJaccardDist(cdBinary)
    FinalPCAData <- t(SingleCell.Binary.Jaccard)
    PCx=1
    PCy=2
    pcaPRComp <- prcomp(FinalPCAData)
    percentVar <- pcaPRComp$sdev^2/sum(pcaPRComp$sdev^2)
 
    if(is.null(groups)){
        df<-data.frame(PCX=pcaPRComp$x[,PCx],PCY=pcaPRComp$x[,PCy])
        p1<-ggplot(df, aes_string(x="PCX",y ="PCY"))        
    }
    else{
        df<-data.frame(PCX=pcaPRComp$x[,PCx],PCY=pcaPRComp$x[,PCy], groups=groups)
        p1<-ggplot(df, aes_string(x="PCX",y ="PCY", color="groups"))             
    }          
    if(missing(title))
        p1<-p1+ggtitle("PCA with Jacard Matrix")
    else
        p1<-p1+ggtitle(title)
        
    p1<-p1+geom_point(size = 3)
    p1<-p1+xlab(paste("PC",PCx,": ", round(percentVar[PCx] * 100), "% variance"))
    p1<-p1+ylab(paste("PC",PCy,": ", round(percentVar[PCy] * 100), "% variance"))
    if(text.label == TRUE){
        p1<-p1+geom_text(data=df,aes(label=cellName),
                       alpha=0.5,size=4, vjust=1,hjust=0.5,angle=45, color="black")
    }
    
    p1<-p1 +
    theme_light(base_size=15)+     
         theme(plot.title = element_text(hjust = 0.5),
              panel.border     = element_blank())

    if(length(levels(groups)) < 12)
        p1 <- p1 + scale_colour_brewer(palette="Set1")
    
    print(p1)
        
    if(ret.val == TRUE)
        return(pcaPRComp)
}

`plotPCAJaccard()`:
> This function calculates the Principle Component Analysis (PCA) and plots the cells based on Jaccard distance in a reduced dimension to visualize how the cells cluster together.

Arguements:
> `cdBinary`: The peak information converted into binary. Ideally this would be file that set in the `outputPeakFileName`. `cdBinary` would be used to calculate the Jaccard distance which would then be used as d matrix in MDS calculation<br>
`groups`: If the cells can be organized into different groups (for eg. different batches), then they are passed as groups making them factor variable.<br>
`ret.val`: Need to set this TRUE for the function to return the MDS values <br>
`text.label`: Need to set this TRUE if label of each of the cells are to be printed in the plot.<br>
`title`: Sets the title of the plot.


Value:
> Calculates PCA and plots the cells in reduced dimension. If`ret.val` is set to TRUE it returns the PCA components along with standard deviations. 

In [19]:
%%R
plotMultiplePCAJaccard <- function(cdBinary, nPCAToDisplay, groups=NULL, ret.val=FALSE , text.label=FALSE, title=""){
    
    if(missing(nPCAToDisplay)){
    	stop("ERROR: Number of PCA's \"nPCAToDisplay\" to display is missing")
    }
    if(missing(ret.val)){
        ret.val = FALSE
    }
    
    SingleCell.Binary.Jaccard <- getJaccardDist(cdBinary)
    FinalPCAData <- t(SingleCell.Binary.Jaccard)
    PCx=1
    PCy=2
    pcaPRComp <- prcomp(FinalPCAData)

    nmax = nPCAToDisplay
    plotTitle = paste0('Plotting first ',nmax,' PCAs')
    
    txt1 <- paste("Percent_PC_Var_onfirst",nmax,"PCs",sep="")
    pca_var = pcaPRComp$sdev ^ 2
    pca_var_percent <- 100 * pca_var / sum(pca_var)
    pca_var_percent_first10 <- NA * pca_var
    pca_var_percent_first10[1:nmax] <- 100 * pca_var[1:nmax] / sum(pca_var[1:nmax])

    pca_var_out <- data.frame(round(pca_var,3),round(pca_var_percent,1),
                          round(pca_var_percent_first10,1))
    rownames(pca_var_out) <- colnames(pcaPRComp$x)
    colnames(pca_var_out) <- c("PC_Var","PC_Var_percent",txt1)


    if(is.null(groups)){
        df <- as.data.frame(pcaPRComp$x)
        p <- ggpairs(df, columns=1:nPCAToDisplay, upper=list(continuous="points"), 
             title=plotTitle,              
             columnLabels = as.character(paste0(colnames(df[,1:nPCAToDisplay]), ' : ', 
                                                pca_var_out$PC_Var_percent[1:nPCAToDisplay], '% variance')))+
    	     theme_light(base_size=15)+     
             theme(plot.title = element_text(hjust = 0.5))        
    }
    else{
        df <- as.data.frame(pcaPRComp$x)
        df$Cell=groups        
        p <- ggpairs(df, columns=1:nPCAToDisplay, upper=list(continuous="points"), 
             title=plotTitle, 
             mapping = aes_string(color="Cell"),
             legend = c(1,nPCAToDisplay),
             columnLabels = as.character(paste0(colnames(df[,1:nPCAToDisplay]), ' : ', 
                                                pca_var_out$PC_Var_percent[1:nPCAToDisplay], '% variance')))+
             theme_light(base_size=15)+     
             theme(plot.title = element_text(hjust = 0.5))
    }          
    
    print(p)
}

`plotMultiplePCAJaccard()`:
> This function calculates the Principle Component Analysis (PCA) based on Jacard distance and plots the first `n` components of the PCA.

Arguements:
> `cdBinary`: The peak information converted into binary. Ideally this would be file that set in the `outputPeakFileName`. `cdBinary` would be used to calculate the Jaccard distance which would then be used as d matrix in MDS calculation<br>
`nPCAToDisplay` : Number of PCAs to display. <br>
`groups`: If the cells can be organized into different groups (for eg. different batches), then they are passed as groups making them factor variable.<br>
`ret.val`: Need to set this TRUE for the function to return the MDS values <br>
`text.label`: Need to set this TRUE if label of each of the cells are to be printed in the plot.<br>
`title`: Sets the title of the plot.


Value:
> Calculates PCA and plots the cells in first `n` components. If`ret.val` is set to TRUE it returns the PCA components along with standard deviations. 

In [20]:
%%R
plotVarExplained <- function(cdBinary, nPCAToDisplay, groups=NULL, ret.val=FALSE , text.label=FALSE, title=""){

    if(missing(nPCAToDisplay)){
    	stop("ERROR: Number of PCA's \"nPCAToDisplay\" to display is missing")
    }
    if(missing(ret.val)){
        ret.val = FALSE
    }
    
    SingleCell.Binary.Jaccard <- getJaccardDist(cdBinary)
    FinalPCAData <- t(SingleCell.Binary.Jaccard)
    PCx=1
    PCy=2
    pcaPRComp <- prcomp(FinalPCAData)

    nmax = nPCAToDisplay
    plotTitle = paste0('Plotting first ',nmax,' PCAs')
    
    txt1 <- paste("Percent_PC_Var_onfirst",nmax,"PCs",sep="")
    pca_var = pcaPRComp$sdev ^ 2
    pca_var_percent <- 100 * pca_var / sum(pca_var)
    pca_var_percent_first10 <- NA * pca_var
    pca_var_percent_first10[1:nmax] <- 100 * pca_var[1:nmax] / sum(pca_var[1:nmax])

    pca_var_out <- data.frame(round(pca_var,3),round(pca_var_percent,1),round(pca_var_percent_first10,1))
    rownames(pca_var_out) <- colnames(pcaPRComp$x)
    colnames(pca_var_out) <- c("PC_Var","PC_Var_percent","PC_Var_n_percent")


    p <- ggplot(pca_var_out[1:nmax,]) + scale_colour_hue(l=40) 
    p <- p + geom_line(aes(x=c(1:nmax), y = PC_Var_percent, color="PC_Var_percent"), size=1.5)+
         geom_point(aes(x=c(1:nmax), y = PC_Var_percent, color="PC_Var_percent"), size = 2)+
         geom_line(aes(x=c(1:nmax), y = PC_Var_n_percent, color="PC_Var_n_percent"), size=1.5)+
         geom_point(aes(x=c(1:nmax), y = PC_Var_n_percent, color="PC_Var_n_percent"), size = 2)+
         ggtitle("Total variance by the PCAs considered")+
         xlab(paste("PC1 to PC10"))+
         ylab(paste("% of Variance"))+     
         theme_light(base_size=15)+     
         theme(plot.title = element_text(hjust = 0.5),
              panel.border     = element_blank())+
         scale_x_continuous(breaks=c(1:nmax))+
         scale_colour_brewer(palette="Set1")
         
   print(p)
}

`plotVarExplained()`:
> This function calculates the percent of variance explained by the PCAs and plots it.

Arguements:
> `cdBinary`: The peak information converted into binary. Ideally this would be file that set in the `outputPeakFileName`. `cdBinary` would be used to calculate the Jaccard distance which would then be used as d matrix in MDS calculation<br>
`nPCAToDisplay` : Number of PCAs to display. <br>
`groups`: If the cells can be organized into different groups (for eg. different batches), then they are passed as groups making them factor variable.<br>
`ret.val`: Need to set this TRUE for the function to return the MDS values <br>
`text.label`: Need to set this TRUE if label of each of the cells are to be printed in the plot.<br>
`title`: Sets the title of the plot.


Value:
> Calculates the percent of variance explained by the PCAs and the plots thsoe percentage. 

In [21]:
%%R
plot_tSNE <- function(cdBinary, nPCAToUSE, groups=NULL, cellName, perplexity_division, ret.val=FALSE , text.label=FALSE, title=""){

    if(missing(nPCAToUSE)){
    	stop("ERROR: Number of PCA's \"nPCAToUSE\" to use is missing")
    }
    if(missing(ret.val)){
        ret.val = FALSE
    }
    
    if(text.label==TRUE & missing(cellName)){
        stop("ERROR: Please give cellName")
    }
    
    if(missing(perplexity_division)){
        stop("ERROR: Please enter the number with which you want to divide the dimension of your data for perplexity setting")
    }
        
    
    SingleCell.Binary.Jaccard <- getJaccardDist(cdBinary)
    FinalPCAData <- t(SingleCell.Binary.Jaccard)

    pcaPRComp <- prcomp(FinalPCAData)

    
    rtsne_pca_out <- Rtsne(as.matrix(pcaPRComp$x[,1:nPCAToUSE]), perplexity = dim(pcaPRComp$x)[1]/perplexity_division, check_duplicates = FALSE)
    
    if(is.null(groups)){
        if(ret.val==TRUE)
            df<-data.frame(X=rtsne_pca_out$Y[,1],Y=rtsne_pca_out$Y[,2], cellName=cellName)
        else
            df<-data.frame(X=rtsne_pca_out$Y[,1],Y=rtsne_pca_out$Y[,2])
        p1 <- ggplot(df, aes_string(x="X",y ="Y"))
    }
    else{
        if(ret.val==TRUE)
            df<-data.frame(X=rtsne_pca_out$Y[,1],Y=rtsne_pca_out$Y[,2], Cell=colnames(SingleCell.Binary.Jaccard), Batch=groups, cellName=cellName)
        else
            df<-data.frame(X=rtsne_pca_out$Y[,1],Y=rtsne_pca_out$Y[,2], Cell=colnames(SingleCell.Binary.Jaccard), Batch=groups)
        p1 <- ggplot(df, aes_string(x="X",y ="Y", color="Batch"))
    }
    

    p1<-p1 + ggtitle("t-SNE plot")
    p1<-p1 + geom_point(size = 2) 
    p1<-p1 + xlab(paste("Dim-1"))
    p1<-p1 + ylab("Dim-2")+
        theme_light(base_size=15) +
        theme(strip.background = element_blank(),
              panel.border     = element_blank(),
              plot.title = element_text(hjust = 0.5))
    if(length(levels(groups)) < 12)
        p1 <- p1 + scale_colour_brewer(palette="Set1")
    if(text.label==TRUE)
        p1<-p1 + geom_text(data=df,aes(label=cellName),alpha=0.5,size=4, vjust=1,hjust=0.5,angle=45)
    print(p1)
    return(df)
}

`plot_tSNE()`:
> Caculates the t-distributed stochastic neighbor embedding (t-SNE) which is a non-linear dimensionality reduction technique particualrly suitable for embedding high-dimensional data into a space of two or three dimensions. This function then plots the lower dimensional representation of the cells in a 2-D scatter plot.

Arguements:
> `cdBinary`: The peak information converted into binary. Ideally this would be file that set in the `outputPeakFileName`. `cdBinary` would be used to calculate the Jaccard distance which would then be used as d matrix in MDS calculation<br>
`nPCAToDisplay` : Number of PCAs to be used to calcualte the t-SNE values. <br>
`groups`: If the cells can be organized into different groups (for eg. different batches), then they are passed as groups making them factor variable.<br>
`ret.val`: Need to set this TRUE for the function to return the MDS values <br>
`text.label`: Need to set this TRUE if label of each of the cells are to be printed in the plot.<br>
`title`: Sets the title of the plot.


Value:
> Calculates and plots the t-SNE representation of the cells. 

In [22]:
%%R
getDiffAccess <- function(cdBinary, groups=NULL){

    if (length(levels(groups)) != 2) {
        stop(paste("ERROR: wrong number of levels in the grouping factor (", 
            paste(levels(groups), collapse = " "), "), but must be two.", 
            sep = ""))
    }
    if (is.null(groups)){
        stop("ERROR: groups factor is not provided")
    }
    
    
    SingleCell.Group1.CellNames <- names(groups[groups==levels(groups)[1]])
    SingleCell.Group2.CellNames <- names(groups[groups==levels(groups)[2]])
    
    SingleCell.Group1.Binary <- cdBinary[,SingleCell.Group1.CellNames]
    SingleCell.Group2.Binary <- cdBinary[,SingleCell.Group2.CellNames]

    SingleCell.Group1.ZeroCount <- rowSums(SingleCell.Group1.Binary==0)
    SingleCell.Group2.ZeroCount <- rowSums(SingleCell.Group2.Binary==0)
    SingleCell.Group1.OneCount <- rowSums(SingleCell.Group1.Binary==1)
    SingleCell.Group2.OneCount <- rowSums(SingleCell.Group2.Binary==1)
    
    SingleCell.Group1VsGroup2 <- data.frame(Chr = cdBinary[,1],
                                           Start = cdBinary[,2],
                                           end = cdBinary[,3])

    SingleCell.Group1VsGroup2$group1OneCounts <- SingleCell.Group1.OneCount
    SingleCell.Group1VsGroup2$group2OneCounts <- SingleCell.Group2.OneCount
    SingleCell.Group1VsGroup2$log2Fold <- log2(SingleCell.Group1.OneCount+1) - log2(SingleCell.Group2.OneCount+1)
    pvalue = vector(mode="numeric", length=length(SingleCell.Group2.OneCount))
    nGenes <- nrow(SingleCell.Group1.Binary)
    
    
    for(i in 1:length(SingleCell.Group2.OneCount))
    {
        contingencyTable <- matrix(c(SingleCell.Group1.ZeroCount[i], SingleCell.Group2.ZeroCount[i],
                                     SingleCell.Group1.OneCount[i], SingleCell.Group2.OneCount[i]), ncol=2)
        SingleCell.Group1VsGroup2$pvalue[i] <- fisher.test(contingencyTable)$p.value        
    }
    
    # Bonferroni correction
    SingleCell.Group1VsGroup2$p_adjust <- p.adjust(SingleCell.Group1VsGroup2$pvalue, method="bonferroni")

    write.csv(SingleCell.Group1VsGroup2[order(SingleCell.Group1VsGroup2$pvalue, decreasing=FALSE),], 
              paste0(levels(groups)[1],'_vs_',levels(groups)[2],'.csv'), row.names=FALSE)
    
    return(SingleCell.Group1VsGroup2)    
}

`getDiffAccess()`:
> Caculates differential accessibility between two groups of cells. For each peak it creates a contingency table with number of 0's and 1's for the two groups. <br>

>|               	| Group-1 	| Group-2 	|
|---------------	|---------	|---------	|
| Number of 0's 	|         	|         	|
| Number of 1's 	|         	|         	| 
<br>
It then runs Fisher exact test to calculate the p-values. These p-values are then corrected for multiple testing with Bonferroni correction. Finally the function returns a table with the peaks, their count of 1's in each group, the p-value and the q-value (corrected p-value).

Arguements:
> `cdBinary`: The peak information converted into binary. Ideally this would be file that set in the `outputPeakFileName`. `cdBinary` would be used to calculate the Jaccard distance which would then be used as d matrix in MDS calculation<br>

Value:
> Returns a table with the peaks, their count of 1's in each group, the p-value and the q-value (corrected p-value).

In [23]:
%%R
MotifMatches <- function(cd, outputFile='cdMotif.csv', species = "Homo sapiens", collection = "CORE", ...){

    # Setting the parameters
    option <- list()
    option["species"] <- species
    option["collection"] <- collection
    option <- c(option, list(...))
    # Finding the motifs
    jaspar_motifs <- TFBSTools::getMatrixSet(JASPAR2016::JASPAR2016, option)
    if (!isTRUE(all.equal(TFBSTools::name(jaspar_motifs), names(jaspar_motifs)))) 
        names(jaspar_motifs) <- paste(names(jaspar_motifs), TFBSTools::name(jaspar_motifs), sep = "_")

    #jaspar_motifs <- get_jaspar_motifs()

    cdGenomicRangeObj <- makeGRangesFromDataFrame(cd,
                         keep.extra.columns=FALSE,
                         ignore.strand=TRUE,
                         seqinfo=NULL,
                         seqnames.field=c("seqnames"),
                         start.field="start",
                         end.field=c("end"),
                         strand.field="strand",
                         starts.in.df.are.0based=FALSE)
    motif_ix <- motifmatchr::match_motifs(jaspar_motifs, cdGenomicRangeObj, out="matches")

    motifMatch <- as.data.frame(as.matrix(unlist(motif_ix@assays$data)))
    motifMatch <- 1*motifMatch
    rownames(motifMatch) <- rownames(motif_ix)
    colnames(motifMatch) <- colnames(motif_ix)
    
    FinalMotifList <- motifMatch[rowSums(motifMatch)>0,]

    cdMod <- cd[rownames(FinalMotifList),]
    write.csv('cdMod',outputFile)
    
    return(cdMod)    
   
} 

`MotifMatches()`:
> This function matches with the motifs in JASPAR database and the given peaks and reports a matrix with the peaks that has atleast one motif intersected.

Arguements:
> `cd`: A binay matrix with the peak accessiblity of all the single-cells. The row would reprsent each of the peaks unique name and the columns would be each cell. Each entry in the matrix is 0/1 representing either the peak is close/open for that cell. <br>
> `species`: Which species the motifs represent?  use eithe jaspar code or latin name.  default is 'Homo sapiens' . <br>
> `outputFile` : Where the results would be written. By default it would be written in cdMotif.csv file along with the accessibility. <br>

Value:
> A matrix that intersects with the motifs of that species.

In [24]:
%%R
clean.count.peaks <- function(cd, min.cell.peaks.obs = 3, min.peaks.cell = 10){
    cdFilt <- cd[,1:3]
    cdCells <- cd[,4:dim(cd)[2]]
    valid.cell.ix <- which(colSums(cdCells) >= min.peaks.cell)
    cdFilt <- cbind(cdFilt,cdCells[,valid.cell.ix])
    cdFilt <- cdFilt[rowSums(cdFilt[,4:dim(cdFilt)[2]]) >=min.cell.peaks.obs ,]
    return(cdFilt)
}

`clean.count.peaks()`:
> This function filter peaks and cells based on peak and cell requirements.

Usage:
> `clean.count.peaks(cd, min.cell.peaks.obs = 3, min.peaks.cell = 10)`

Arguements:
> `cd`: A binay matrix with the peak accessiblity of all the single-cells. The row would reprsent each of the peaks unique name and the columns would be each cell. Each entry in the matrix is 0/1 representing either the peak is close/open for that cell. <br>
> `min.cell.peaks.obs`: Minimum number of cells that a peak has to be observed in order for that peak to be a valid peak. <br>
> `min.peaks.cell` : Minimum number of peaks that a cell needs to have in order for that cell to be called a valid cell. <br>

Value:
> A matrix with valid accessible peaks.

--------------

### Configuration Cell
Following is the cell where we configure all the parameters for the tool.

In [25]:
# This cell holds the parameters for the tools

# Set this parameter to the folder where all the Peak Files are. This has to be the folder
# where each of the individual cells peak files reside
peakFolder = '/home/baker/my-dpsf-scratch-share/Buenrostro_pipeline/Human_data_analysis_Second_Folder/output/Filtered_Macs2_files'


# Set this parameter to the folder where the peaks for the merge files are. This can be the same folder
# of all the individual cells peak folder or can be a single peak file that is generated from merging BAM files
peakFolderToMerge = '/home/baker/my-dpsf-scratch-share/Buenrostro_pipeline/Human_data_analysis_Second_Folder/output/Bam_Files_Filtered_on_Library_Size/'



# Set up the pattern for the BAM file ending
BAMFilePattern = '_nodup_sorted_cleaned.bam'

# Set this parameter to the pattern that is common to all the peak file names that you wish to merge. 
# for eg, if you want to combine OE19_C01_peaks.narrowPeak and OE19_C02_peaks.narrowPeak then the
# pattern would be '_peaks.narrowPeak'
# Allowed patterns while calling with the pipeline provided here are
# _summits_shifted.bed and '_peaks.narrowPeak'
peakFilePattern = '_peaks.narrowPeak'

# This would be automatically set to 'mergedPeaks.bed' file in peakFolder folder
mergedPeakFile = '/home/baker/my-dpsf-scratch-share/Buenrostro_pipeline/Human_data_analysis_Second_Folder/output/Bam_Files_Filtered_on_Library_Size/mergedPeaks.bed'

# Set this folder to where the PeakHits.csv File would be saved
outputFolder = '/home/baker/my-dpsf-scratch-share/Buenrostro_pipeline/Human_data_analysis_Second_Folder/output/Bam_Files_Filtered_on_Library_Size/'

# Set the outputPeakFileName name here
outputPeakFileName = 'All_Human_Bam.csv'

# This will be automatically set to the 'PeakHits.csv' file in outputFolder folder
peakFile = outputFolder + outputPeakFileName

We now call `getAggregatedPeak()` to merge all the BAM files and generated the aggregated peak from the merged BAM file.

In [55]:
getAggregatedPeak(peakFolderToMerge, BAMFilePattern)

To calculates the accessiblity of each of the peaks for all the cells we call the following function.

In [56]:
%%R -i mergedPeakFile,peakFolder,peakFilePattern,outputFolder,outputPeakFileName
peakAccesibility(mergedPeakFile, peakFolder, peakFilePattern, outputFolder, outputPeakFileName)

Reading the peak file

In [26]:
%%R -i peakFile
cd <- read.csv(peakFile, sep=' ', header=TRUE)

## Filtering accessible peaks

In [27]:
%%R
cdFilt <- clean.count.peaks(cd)
print(dim(cd))
print(dim(cdFilt))

[1] 113939    574
[1] 62536   562


In [28]:
%%R
write.csv(cdFilt, 'FilteredPeakCounts.csv')